# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [2]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [3]:
raw_df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [8]:
raw_df['combined_text'] = raw_df['headline'] + ' ' + raw_df['short_description']
raw_df.head()

,category,headline,authors,link,short_description,date,combined_text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...


In [9]:
data = list(map(word_tokenize, raw_df['combined_text']))

Inspect the first 5 items in `data` to see how everything looks. 

In [10]:
data[:5]

[['There',
  'Were',
  '2',
  'Mass',
  'Shootings',
  'In',
  'Texas',
  'Last',
  'Week',
  ',',
  'But',
  'Only',
  '1',
  'On',
  'TV',
  'She',
  'left',
  'her',
  'husband',
  '.',
  'He',
  'killed',
  'their',
  'children',
  '.',
  'Just',
  'another',
  'day',
  'in',
  'America',
  '.'],
 ['Will',
  'Smith',
  'Joins',
  'Diplo',
  'And',
  'Nicky',
  'Jam',
  'For',
  'The',
  '2018',
  'World',
  'Cup',
  "'s",
  'Official',
  'Song',
  'Of',
  'course',
  'it',
  'has',
  'a',
  'song',
  '.'],
 ['Hugh',
  'Grant',
  'Marries',
  'For',
  'The',
  'First',
  'Time',
  'At',
  'Age',
  '57',
  'The',
  'actor',
  'and',
  'his',
  'longtime',
  'girlfriend',
  'Anna',
  'Eberstein',
  'tied',
  'the',
  'knot',
  'in',
  'a',
  'civil',
  'ceremony',
  '.'],
 ['Jim',
  'Carrey',
  'Blasts',
  "'Castrato",
  "'",
  'Adam',
  'Schiff',
  'And',
  'Democrats',
  'In',
  'New',
  'Artwork',
  'The',
  'actor',
  'gives',
  'Dems',
  'an',
  'ass-kicking',
  'for',
  'not',
 

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [11]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [12]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55568556, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [13]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [14]:
wv.most_similar('Texas')

[('Louisiana', 0.830621600151062),
 ('Oklahoma', 0.8161239624023438),
 ('Maryland', 0.8159974813461304),
 ('Pennsylvania', 0.8059967756271362),
 ('Illinois', 0.7873678207397461),
 ('Arkansas', 0.7868865728378296),
 ('Florida', 0.7861232757568359),
 ('Ohio', 0.7815952301025391),
 ('Georgia', 0.7787268757820129),
 ('Massachusetts', 0.7776572108268738)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [15]:
wv.most_similar(negative='Texas')

[('Likeable', 0.42309141159057617),
 ('Headstrong', 0.4228079617023468),
 ('uh-oh', 0.42100071907043457),
 ('Sis-In-Law', 0.37501466274261475),
 ('writer-producer', 0.36570245027542114),
 ('Parent/Grandparent', 0.36523735523223877),
 ('Habitable', 0.36209824681282043),
 ('Hunger-Free', 0.36110424995422363),
 ('Compel', 0.35852348804473877),
 ('Strategist™', 0.3540639877319336)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [16]:
wv['Texas']

array([ 2.0980785 , -0.33158153, -0.12609781, -0.4637886 , -0.75266236,
        1.2865642 , -1.7821869 ,  0.41678926,  0.6244918 ,  0.278757  ,
       -0.33648118, -2.1892776 ,  2.0758069 , -0.35101303,  1.1323518 ,
        0.03058979,  0.5087857 ,  0.29824865, -0.5194162 , -0.8392203 ,
        1.3142842 , -0.51835746, -2.5143454 ,  0.07223639, -0.8356729 ,
        1.5485003 , -1.3907362 , -1.2554688 ,  1.0043914 , -1.7070214 ,
       -0.9772619 , -1.1564877 ,  0.49536848, -0.36561573,  2.1086712 ,
        0.6722096 , -1.0656515 , -0.5531165 ,  0.5710325 ,  0.40095195,
        1.6609126 ,  0.64141244, -0.70246017,  3.5544202 ,  0.14753164,
        1.0961245 , -2.373388  , -0.68783456, -0.446178  , -3.265121  ,
       -0.58560514,  0.4463796 , -2.9820662 ,  0.6521489 , -0.32485938,
        0.6115925 , -0.7761048 ,  0.63755   ,  1.8471022 ,  1.1298958 ,
        1.2457175 ,  2.5531137 ,  1.2009187 ,  0.44603318,  2.2767394 ,
       -0.18380542, -0.6784027 , -0.77470404,  1.8985144 , -1.20

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [17]:
wv.vectors

array([[-2.7351686e-01,  3.5819358e-01,  7.5372291e-01, ...,
        -4.4155124e-01, -5.5344647e-01, -9.4181633e-01],
       [-5.4966074e-01,  8.5896686e-02, -4.5398256e-01, ...,
         1.2921782e+00, -1.3632373e+00, -4.2052138e-01],
       [ 9.4026662e-02,  1.3069938e-01, -5.3443086e-01, ...,
         9.0745467e-01, -5.4950494e-02, -2.1086490e+00],
       ...,
       [ 1.6287841e-02, -4.7992361e-03,  9.5050417e-02, ...,
         1.0201327e-01,  1.2184987e-01,  5.0473046e-02],
       [-4.0777419e-03, -6.6791107e-03, -2.8021615e-03, ...,
        -1.8666487e-02,  2.0270061e-02,  3.1172633e-02],
       [ 1.8732153e-02,  2.5470126e-02,  8.5094012e-02, ...,
         9.2254989e-02,  1.7537050e-03,  5.2349925e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [18]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('brunette', 0.588446319103241),
 ('princess', 0.556723415851593),
 ('revival', 0.5564587116241455),
 ('last-chance', 0.5453534722328186),
 ('title', 0.5440288782119751),
 ('queen', 0.5427923202514648),
 ('symbol', 0.5332902669906616),
 ('lover', 0.5301950573921204),
 ('feminist', 0.5247619152069092),
 ('gem', 0.5228767395019531)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!